In [1]:
import os
from dotenv import load_dotenv
from dotenv import dotenv_values
import json

In [2]:
OPENAI_API_VERSION ='2023-07-01-preview'
ENV_FILE_PATH = "../.env"
load_dotenv(ENV_FILE_PATH)
values_env = dotenv_values(ENV_FILE_PATH)
os.environ["AZURE_OPENAI_API_KEY"] =values_env['AZURE_OPENAI_KEY']
os.environ["AZURE_OPENAI_ENDPOINT"] =values_env['AZURE_OPENAI_ENDPOINT']
os.environ["OPENAI_API_VERSION"] =OPENAI_API_VERSION
deployment = values_env['AZURE_OPENAI_DEPLOYMENT_ID_AGENTS']

In [3]:
from langchain.chat_models import AzureChatOpenAI

In [4]:
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [5]:
llm_chat = AzureChatOpenAI(azure_deployment=deployment,temperature=0)

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureChatOpenAI instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [6]:
@tool("AddNumbers")
def add_two_numbers(num1, num2):
    """Function that adds 2 numbers."""""
    return json.dumps({"result": num1 + num2})

@tool("MultiplyNumbers")
def multiply_two_numbers(num1, num2):
    """Function that multiplies 2 numbers."""""
    return json.dumps({"result": num1 * num2})

In [7]:
tools = [add_two_numbers, multiply_two_numbers]

In [8]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])


In [9]:
agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())

In [10]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
action_input = "Add two numbers 1 and 2 and multiply the result by 23."

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `AddNumbers` with `{'num1': 1, 'num2': 2}`


{"result": 3}
Invoking: `MultiplyNumbers` with `{'num1': 3, 'num2': 23}`


{"result": 69}The result of adding 1 and 2 is 3. Multiplying the result by 23 gives us 69.

> Finished chain.


{'input': 'Add two numbers 1 and 2 and multiply the result by 23.',
 'output': 'The result of adding 1 and 2 is 3. Multiplying the result by 23 gives us 69.'}

In [12]:
action_input = "Multiply the 4 by 23."

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `MultiplyNumbers` with `{'num1': 4, 'num2': 23}`


{"result": 92}The result of multiplying 4 by 23 is 92.

> Finished chain.


{'input': 'Multiply the 4 by 23.',
 'output': 'The result of multiplying 4 by 23 is 92.'}

In [13]:
action_input = "Add the numbers 4 by 23."

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `AddNumbers` with `{'num1': 4, 'num2': 23}`


{"result": 27}The sum of 4 and 23 is 27.

> Finished chain.


{'input': 'Add the numbers 4 by 23.', 'output': 'The sum of 4 and 23 is 27.'}

In [14]:
action_input = "Multiply the 4 by 23."

response = agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `MultiplyNumbers` with `{'num1': 4, 'num2': 23}`


{"result": 92}The result of multiplying 4 by 23 is 92.

> Finished chain.


In [15]:
response

{'input': 'Multiply the 4 by 23.',
 'output': 'The result of multiplying 4 by 23 is 92.'}

Chat History

In [16]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [17]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [18]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
action_input = "Multiply the 4 by 23."

response = response = agent_executor.invoke({"input": action_input, "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `MultiplyNumbers` with `{'num1': 4, 'num2': 23}`


{"result": 92}The product of 4 multiplied by 23 is 92.

> Finished chain.


In [20]:
response

{'input': 'Multiply the 4 by 23.',
 'chat_history': [],
 'output': 'The product of 4 multiplied by 23 is 92.'}

In [21]:
chat_history.extend(
    [
        HumanMessage(content=action_input),
        AIMessage(content=response["output"]),
    ]
)

In [22]:
action_input = "Add the numbers 4 by 23."

response = agent_executor.invoke({"input": action_input,"chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `AddNumbers` with `{'num1': 4, 'num2': 23}`


{"result": 27}The sum of 4 and 23 is 27.

> Finished chain.


In [23]:
response["chat_history"][0].content

'Multiply the 4 by 23.'

In [24]:
response["chat_history"][1].content

'The product of 4 multiplied by 23 is 92.'

In [25]:
chat_history.extend(
    [
        HumanMessage(content=action_input),
        AIMessage(content=response["output"]),
    ]
)

In [26]:
action_input = "Add the numbers 13 by 23."

response = agent_executor.invoke({"input": action_input,"chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `AddNumbers` with `{'num1': 13, 'num2': 23}`


{"result": 36}The sum of 13 and 23 is 36.

> Finished chain.


In [27]:
response["chat_history"]

[HumanMessage(content='Multiply the 4 by 23.'),
 AIMessage(content='The product of 4 multiplied by 23 is 92.'),
 HumanMessage(content='Add the numbers 4 by 23.'),
 AIMessage(content='The sum of 4 and 23 is 27.')]